In [ ]:
# Librerias necesarias para hacer funcionar

# pip install pandas numpy openpyxl geopy 

In [ ]:
import pandas as pd
from geopy.distance import geodesic

# Ruta del archivo corregido
ruta_excel = "Horas_trabajo (1).xlsx"

# Cargar la hoja "Loc"
df_loc = pd.read_excel(ruta_excel, sheet_name="Loc", header=1)

In [12]:
df_loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CLIENTE       46 non-null     object
 1   LOCALIDAD     46 non-null     object
 2    PROVINCIA    46 non-null     object
 3   LOCALIZACIÓN  46 non-null     object
dtypes: object(4)
memory usage: 1.6+ KB


In [13]:
df_loc.head()

,CLIENTE,LOCALIDAD,PROVINCIA,LOCALIZACIÓN
0,1,HARO,LA RIOJA,"42.5681060494229, -2.858152511919005"
1,3,STO. DOMINGO,LA RIOJA,"42.433119817282204, -2.950749636445161"
2,4,DURANA,ALAVA,"42.88702626937825, -2.639513912965261"
3,5,HARO,LA RIOJA,"42.56905422294487, -2.8582383426018674"
4,6,VITORIA,ALAVA,"42.83740036472884, -2.6700945850372797"


In [14]:
# Renombrar columnas
df_loc = df_loc.rename(columns={"LOCALIZACIÓN": "COORDS", "CLIENTE ": "CLIENTE_ID"})

In [15]:
# Extraer coordenadas
df_loc["lat"] = df_loc["COORDS"].apply(lambda x: float(str(x).split(",")[0].strip()))
df_loc["lon"] = df_loc["COORDS"].apply(lambda x: float(str(x).split(",")[1].strip()))

# Coordenadas de la sede
coordenadas_sede = (42.5515, -2.8732)

# Calcular tiempo desde sede (asumiendo 50 km/h)
def calcular_tiempo(coor_cliente):
    distancia_km = geodesic(coordenadas_sede, coor_cliente).km
    return (distancia_km / 50) * 60  # minutos

df_loc["tiempo_desde_sede_min"] = df_loc[["lat", "lon"]].apply(lambda x: calcular_tiempo((x["lat"], x["lon"])), axis=1)

# Filtrar clientes válidos (≤ 90 min)
clientes_validos = df_loc[df_loc["tiempo_desde_sede_min"] <= 90]
clientes_validos_ids = clientes_validos["CLIENTE_ID"].astype(str).tolist()

print(f"{len(clientes_validos_ids)} clientes dentro del rango de 90 minutos.")
print("Primeros 5:", clientes_validos_ids[:5])

38 clientes dentro del rango de 90 minutos.
Primeros 5: ['1', '3', '4', '5', '6']


In [19]:
from geopy.distance import geodesic
import pandas as pd

# Extraer lat y lon de la columna 'COORDS'
df_loc["lat"] = df_loc["COORDS"].apply(lambda x: float(str(x).split(",")[0].strip()))
df_loc["lon"] = df_loc["COORDS"].apply(lambda x: float(str(x).split(",")[1].strip()))

# Coordenadas de la sede
coordenadas_sede = (42.5515, -2.8732)

# Función para calcular tiempo desde la sede en minutos (50 km/h)
def calcular_tiempo(coor_cliente):
    distancia_km = geodesic(coordenadas_sede, coor_cliente).km
    return (distancia_km / 50) * 60  # minutos

# Calcular tiempo estimado desde la sede
df_loc["tiempo_desde_sede_min"] = df_loc[["lat", "lon"]].apply(
    lambda x: calcular_tiempo((x["lat"], x["lon"])), axis=1
)

# Crear nuevo DataFrame solo con empresas cercanas (≤ 90 min)
df_clientes_cercanos = df_loc[df_loc["tiempo_desde_sede_min"] <= 90].copy()

# Mostrar resumen
print(f"{len(df_clientes_cercanos)} empresas están dentro del rango de 90 minutos.")
print(df_clientes_cercanos[["CLIENTE_ID", "LOCALIDAD", "tiempo_desde_sede_min"]].head())

df_clientes_cercanos.to_csv('clientes_90min.csv')


38 empresas están dentro del rango de 90 minutos.
  CLIENTE_ID     LOCALIDAD  tiempo_desde_sede_min
0          1          HARO               2.664373
1          3  STO. DOMINGO              17.536715
2          4        DURANA              50.280536
3          5          HARO               2.765751
4          6       VITORIA              43.027608
